Author: Dr. Göktuğ Aşcı <br>
Email: gok.asci@student.ie.edu <br>
Date: 2021-02-21 <br>
Licence: MIT License <br>
Data: [Rain in Australia](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package)

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://www.visitvictoria.com/-/media/images/gippsland/things-to-do/nature-and-wildlife/rainforest-in-orbost_gip_u_1031651_1600x900.jpg?ts=20160906510302")

## Background Information
Rainfall in Australia has great importance since Australia has a huge ecosystem of animals and plants and it is home to many species. Without rain, forests and animals suffer great loss. Results of this loss is beyond Australia and it may have unwanted consequences. Being able to understand contributing factors on rainfall in Australia helps authorities to take precautions before unwanted consequences emerge. <br>

This dataset contains approximately 10 years of daily weather observations from many locations across Australia. <br>

RainTomorrow is the variable that shows if it rained the next day. Yes or No? This column is Yes if the rain for that day was 1mm or more. <br>

We are going to analyze this data to see the contributing factors to the next day’s rainfall  in Australia.

## Goal of the Analysis
We want to analyze the dataset to discover the regions with the highest and lowest levels of rainfall to compare both groups so that we can focus on improving found variables in the regions with the lowest levels of rainfall.

## 1. PySpark environment setup

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql.functions import *

In [4]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.getOrCreate()

## 2. Data source and Spark data abstraction (DataFrame) setup

In [5]:
df = (
    spark
    .read
    .csv('weatherAUS.csv', inferSchema=True, header=True)
    .cache()
) # optimization to make the processing faster

In [6]:
df.columns

['Date',
 'Location',
 'MinTemp',
 'MaxTemp',
 'Rainfall',
 'Evaporation',
 'Sunshine',
 'WindGustDir',
 'WindGustSpeed',
 'WindDir9am',
 'WindDir3pm',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm',
 'Cloud9am',
 'Cloud3pm',
 'Temp9am',
 'Temp3pm',
 'RainToday',
 'RainTomorrow']

## 3. Data set metadata analysis
### A. Display schema and size of the DataFrame / Simple data cleaning

In [7]:
from IPython.display import display, Markdown
from pyspark.sql.types import *

In [8]:
df.printSchema()
"This DataFrame has {} rows and {} columns.".format(df.count(), len(df.columns)) 

root
 |-- Date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: string (nullable = true)
 |-- MaxTemp: string (nullable = true)
 |-- Rainfall: string (nullable = true)
 |-- Evaporation: string (nullable = true)
 |-- Sunshine: string (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: string (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: string (nullable = true)
 |-- WindSpeed3pm: string (nullable = true)
 |-- Humidity9am: string (nullable = true)
 |-- Humidity3pm: string (nullable = true)
 |-- Pressure9am: string (nullable = true)
 |-- Pressure3pm: string (nullable = true)
 |-- Cloud9am: string (nullable = true)
 |-- Cloud3pm: string (nullable = true)
 |-- Temp9am: string (nullable = true)
 |-- Temp3pm: string (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RainTomorrow: string (nullable = true)



'This DataFrame has 145460 rows and 23 columns.'

We have null values in the table but they are represented as NA. First we should convert them to null:

### Simple Data Cleaning and Preparation

In [9]:
def blank_as_null(x):
    return when(col(x) != "NA", col(x)).otherwise(None)

df = (df
      .withColumn("Date", blank_as_null("Date"))
      .withColumn("Location", blank_as_null("Location"))
      .withColumn("MinTemp", blank_as_null("MinTemp").cast(FloatType()))
      .withColumn("MaxTemp", blank_as_null("MaxTemp").cast(FloatType()))
      .withColumn("Rainfall", blank_as_null("Rainfall").cast(FloatType()))
      .withColumn("Evaporation", blank_as_null("Evaporation").cast(FloatType()))
      .withColumn("Sunshine", blank_as_null("Sunshine").cast(FloatType()))
      .withColumn("WindGustDir", blank_as_null("WindGustDir"))
      .withColumn("WindGustSpeed", blank_as_null("WindGustSpeed").cast(IntegerType()))
      .withColumn("WindDir9am", blank_as_null("WindDir9am"))
      .withColumn("WindDir3pm", blank_as_null("WindDir3pm"))
      .withColumn("WindSpeed9am", blank_as_null("WindSpeed9am").cast(IntegerType()))
      .withColumn("WindSpeed3pm", blank_as_null("WindSpeed3pm").cast(IntegerType()))
      .withColumn("Humidity9am", blank_as_null("Humidity9am").cast(IntegerType()))
      .withColumn("Humidity3pm", blank_as_null("Humidity3pm").cast(IntegerType()))
      .withColumn("Pressure9am", blank_as_null("Pressure9am").cast(FloatType()))
      .withColumn("Pressure3pm", blank_as_null("Pressure3pm").cast(FloatType()))
      .withColumn("Cloud9am", blank_as_null("Cloud9am").cast(IntegerType()))
      .withColumn("Cloud3pm", blank_as_null("Cloud3pm").cast(IntegerType()))
      .withColumn("Temp9am", blank_as_null("Temp9am").cast(FloatType()))
      .withColumn("Temp3pm", blank_as_null("Temp3pm").cast(FloatType()))
      .withColumn("RainToday", blank_as_null("RainToday"))
      .withColumn("RainTomorrow", blank_as_null("RainTomorrow"))
)

**Date**: The date of observation <br>
**Location**: The common name of the location of the weather station <br>
**MinTemp**: The minimum temperature in degrees celsius <br>
**MaxTemp**: The maximum temperature in degrees celsius <br>
**Rainfall**: The amount of rainfall recorded for the day in mm <br>
**Evaporation**: The so-called Class A pan evaporation (mm) in the 24 hours to 9am <br>
**Sunshine**: The number of hours of bright sunshine in the day. <br> 
**WindGustDir**: The direction of the strongest wind gust in the 24 hours to midnight <br>
**WindGustSpeed**: The speed (km/h) of the strongest wind gust in the 24 hours to midnight <br> 
**WindDir9am**: Direction of the wind at 9am <br>
**WindDir3pm**: Direction of the wind at 3pm <br>
**WindSpeed9am**: Wind speed (km/hr) averaged over 10 minutes prior to 9am <br>
**WindSpeed3pm**: Wind speed (km/hr) averaged over 10 minutes prior to 3pm <br>
**Humidity9am**: Humidity (percent) at 9am <br>
**Humidity3pm**: Humidity (percent) at 3pm <br>
**Pressure9am**: Atmospheric pressure (hpa) reduced to mean sea level at 9am <br>
**Pressure3pm**: Atmospheric pressure (hpa) reduced to mean sea level at 3pm <br>
**Cloud9am**: Fraction of sky obscured by cloud at 9am. This is measured in "oktas", which are a unit of eigths. It records how many <br>
**Cloud3pm**: Fraction of sky obscured by cloud (in "oktas": eighths) at 3pm. See Cload9am for a description of the values <br>
**Temp9am**: Temperature (degrees C) at 9am <br>
**Temp3pm**: Temperature (degrees C) at 3pm <br>
**RainToday**: True if precipitation (mm) in the 24 hours to 9am exceeds 1mm, otherwise 0 <br>
**RainTomorrow**: The amount of next day rain in mm. Used to create response variable RainTomorrow. A kind of measure of the "risk". 

In [10]:
df.stat.corr("Rainfall","MinTemp")

0.10308227743558884

Relatively high correlation with MinTemp but still low.

In [11]:
df.stat.corr("Rainfall","MaxTemp")

-0.06707040954376242

In [12]:
df.stat.corr("Rainfall","Evaporation")

-0.053597156761932274

In [13]:
df.stat.corr("Rainfall","Sunshine")

-0.09648941553419445

In [14]:
df.stat.corr("Rainfall","WindGustSpeed")

0.09238192821408384

In [15]:
df.stat.corr("Rainfall","WindSpeed9am")

0.08427159460703114

In [16]:
df.stat.corr("Rainfall","WindSpeed3pm")

0.05195824896950231

In [17]:
df.stat.corr("Rainfall","Humidity9am")

0.20025536198537847

In [18]:
df.stat.corr("Rainfall","Humidity3pm")

0.22498815685917117

It looks like Humidity at any hour has the highest correlation with the rainfall compared to other variables.

In [19]:
df.stat.corr("Rainfall","Pressure9am")

-0.000628629675978074

In [20]:
df.stat.corr("Rainfall","Pressure3pm")

-0.0006133432662193096

In [21]:
df.stat.corr("Rainfall","Temp9am")

0.012563916616111816

In [22]:
df.stat.corr("Rainfall","Temp3pm")

-0.07218438816291148

### B. Get one sample from the data set

In [23]:
# take(1)
display(df.take(1))

[Row(Date='2008-12-01', Location='Albury', MinTemp=13.399999618530273, MaxTemp=22.899999618530273, Rainfall=0.6000000238418579, Evaporation=None, Sunshine=None, WindGustDir='W', WindGustSpeed=44, WindDir9am='W', WindDir3pm='WNW', WindSpeed9am=20, WindSpeed3pm=24, Humidity9am=71, Humidity3pm=22, Pressure9am=1007.7000122070312, Pressure3pm=1007.0999755859375, Cloud9am=8, Cloud3pm=None, Temp9am=16.899999618530273, Temp3pm=21.799999237060547, RainToday='No', RainTomorrow='No')]

### C. Data entities, metrics and dimensions

I've identified the following elements:

* **Entities:** Weather
* **Metrics:** RainTomorrow, Rainfall, RainToday, MinTemp, MaxTemp, Evaporation, Sunshine, WindGustSpeed, WindSpeed9am, WindSpeed3pm, Humidity9am, Humidity3pm, Pressure9am, Pressure3pm, Cloud9am, Cloud3pm, Temp9am, Temp3pm
* **Dimensions:** Location (location), Date (time), WindGustDir, WindDir9am, WindDir3pm

### D. Column categorization

The following could be a potential column categorization:

* **Time and Location related columns:** Date, Location
* **Wind related columns:** WindGustDir, WindDir9am, WindDir3pm, WindGustSpeed, WindSpeed9am, WindSpeed3pm
* **Temperature related columns.** MinTemp, MaxTemp, Temp9am, Temp3pm
* **Rain related and other columns.** Humidity9am, Humidity3pm, Pressure9am, Pressure3pm, Cloud9am, Cloud3pm, Evaporation, Sunshine
* **Issue related columns:** RainTomorrow, Rainfall, RainToday

## 4. Columns groups basic profiling to better understand our data set
### A. Timing and location related columns basic profiling

In [24]:
from IPython.display import display, Markdown

In [25]:
# let's add year month day seperately for daily, montly, yearly analysis
split_date = split(df['Date'], '-')     
df = df.withColumn('Year', split_date.getItem(0))
df = df.withColumn('Month', split_date.getItem(1))
df = df.withColumn('Day', split_date.getItem(2))

In [26]:
analysis_col_list = ["Location", "Date" ,"Year" ,"Month" ,"Day"]

print ("Summary of columns Date and Location:")
df.select("Date","Location").summary().show()

Summary of columns Date and Location:
+-------+----------+--------+
|summary|      Date|Location|
+-------+----------+--------+
|  count|    145460|  145460|
|   mean|      null|    null|
| stddev|      null|    null|
|    min|2007-11-01|Adelaide|
|    25%|      null|    null|
|    50%|      null|    null|
|    75%|      null|    null|
|    max|2017-06-25| Woomera|
+-------+----------+--------+



In [27]:
print("Checking for nulls on columns:")
df.select([count(when(col(c).isNull(), c)).alias(c) for c in analysis_col_list]).show()

Checking for nulls on columns:
+--------+----+----+-----+---+
|Location|Date|Year|Month|Day|
+--------+----+----+-----+---+
|       0|   0|   0|    0|  0|
+--------+----+----+-----+---+



Every row includes Location and Date value with (Year, Month, Day given)

In [28]:
print("Checking amount of distinct values:")
df.select([countDistinct(c).alias(c) for c in analysis_col_list]).show()

Checking amount of distinct values:
+--------+----+----+-----+---+
|Location|Date|Year|Month|Day|
+--------+----+----+-----+---+
|      49|3436|  11|   12| 31|
+--------+----+----+-----+---+



We have data from 49 different locations and 3436 days (nearly 10 years)

In [29]:
print ("Most and least frequent occurrences:")
column_name = "Location"

occurencesDF= df.groupBy(column_name).count()
occurencesDF.sort(occurencesDF["count"].desc()).show(10, False)

leastFreq    = occurencesDF.orderBy(col("count").asc()).first()
mostFreq      = occurencesDF.orderBy(col("count").desc()).first()

(
    "leastFreq: {} - {}".format(leastFreq[column_name], leastFreq["count"]),
    "mostFreq: {} - {}".format(mostFreq[column_name], mostFreq["count"]),
)

Most and least frequent occurrences:
+----------+-----+
|Location  |count|
+----------+-----+
|Canberra  |3436 |
|Sydney    |3344 |
|Brisbane  |3193 |
|Darwin    |3193 |
|Adelaide  |3193 |
|Perth     |3193 |
|Hobart    |3193 |
|Melbourne |3193 |
|Wollongong|3040 |
|Cairns    |3040 |
+----------+-----+
only showing top 10 rows



('leastFreq: Nhil - 1578', 'mostFreq: Canberra - 3436')

Nhil has the lowest number of data and Canberra has the highest number

### B. Wind related columns basic profiling

In [30]:
analysis_col_list = ["WindGustDir", "WindDir9am", "WindDir3pm", "WindGustSpeed", "WindSpeed9am", "WindSpeed3pm"]

print ("Summary of columns:")
df.select(analysis_col_list).summary().toPandas()

Summary of columns:


,summary,WindGustDir,WindDir9am,WindDir3pm,WindGustSpeed,WindSpeed9am,WindSpeed3pm
0,count,135134,134894,141232,135197,143693,142398
1,mean,None,None,None,40.03523007167319,14.043425914971502,18.662656778887342
2,stddev,None,None,None,13.60706226738142,8.91537532267949,8.809800021251455
3,min,E,E,E,6,0,0
4,25%,None,None,None,31,7,13
5,50%,None,None,None,39,13,19
6,75%,None,None,None,48,19,24
7,max,WSW,WSW,WSW,135,130,87


The East doesn't blow the wind so often where West-South-West blows the most wind.

In [31]:
print("Checking for nulls on columns:")
df.select([count(when(col(c).isNull(), c)).alias(c) for c in analysis_col_list]).show()

Checking for nulls on columns:
+-----------+----------+----------+-------------+------------+------------+
|WindGustDir|WindDir9am|WindDir3pm|WindGustSpeed|WindSpeed9am|WindSpeed3pm|
+-----------+----------+----------+-------------+------------+------------+
|      10326|     10566|      4228|        10263|        1767|        3062|
+-----------+----------+----------+-------------+------------+------------+



In [32]:
print("Checking amount of distinct values:")
df.select([countDistinct(c).alias(c) for c in analysis_col_list]).show()

Checking amount of distinct values:
+-----------+----------+----------+-------------+------------+------------+
|WindGustDir|WindDir9am|WindDir3pm|WindGustSpeed|WindSpeed9am|WindSpeed3pm|
+-----------+----------+----------+-------------+------------+------------+
|         16|        16|        16|           67|          43|          44|
+-----------+----------+----------+-------------+------------+------------+



As expected the are 4 * 4 * 4 directions

In [33]:
print ("Most and least frequent occurrences:")
column_name = "WindGustSpeed"

occurencesDF= df.groupBy(column_name).count()
occurencesDF.sort(occurencesDF["count"].desc()).show(10, False)

leastFreq    = occurencesDF.orderBy(col("count").asc()).first()
mostFreq      = occurencesDF.orderBy(col("count").desc()).first()

(
    "leastFreq: {} - {}".format(leastFreq[column_name], leastFreq["count"]),
    "mostFreq: {} - {}".format(mostFreq[column_name], mostFreq["count"]),
)

Most and least frequent occurrences:
+-------------+-----+
|WindGustSpeed|count|
+-------------+-----+
|null         |10263|
|35           |9215 |
|39           |8794 |
|31           |8428 |
|37           |8047 |
|33           |7933 |
|41           |7369 |
|30           |7038 |
|43           |6609 |
|28           |6478 |
+-------------+-----+
only showing top 10 rows



('leastFreq: 6 - 1', 'mostFreq: None - 10263')

We can clearly observe that extreme values are less frequent.

### C. Temperature related columns basic profiling

In [34]:
analysis_col_list = ["MinTemp", "MaxTemp", "Temp9am", "Temp3pm"]

print ("Summary of columns:")
df.select(analysis_col_list).summary().toPandas()

Summary of columns:


,summary,MinTemp,MaxTemp,Temp9am,Temp3pm
0,count,143975,144199,143693,141851
1,mean,12.194034381779941,23.221348273321002,16.990631418377568,21.68339031665222
2,stddev,6.39849497591284,7.119048841492777,6.488753139523496,6.936650457604387
3,min,-8.5,-4.8,-7.2,-5.4
4,25%,7.6,17.9,12.3,16.6
5,50%,12.0,22.6,16.7,21.1
6,75%,16.9,28.2,21.6,26.4
7,max,33.9,48.1,40.2,46.7


As expected the mean of Temp3pm is highter than Temp9am (And it is very close the MaxTemp)

In [35]:
print("Checking for nulls on columns:")
df.select([count(when(col(c).isNull(), c)).alias(c) for c in analysis_col_list]).show()

Checking for nulls on columns:
+-------+-------+-------+-------+
|MinTemp|MaxTemp|Temp9am|Temp3pm|
+-------+-------+-------+-------+
|   1485|   1261|   1767|   3609|
+-------+-------+-------+-------+



In [36]:
print("Checking amount of distinct values:")
df.select([countDistinct(c).alias(c) for c in analysis_col_list]).show()

Checking amount of distinct values:
+-------+-------+-------+-------+
|MinTemp|MaxTemp|Temp9am|Temp3pm|
+-------+-------+-------+-------+
|    389|    505|    441|    502|
+-------+-------+-------+-------+



In [37]:
print ("Most and least frequent occurrences:")
column_name = "MinTemp"
column_two_name = "MaxTemp"

occurencesDF= df.groupBy(column_name).count()
occurencesMaxTempDF= df.groupBy(column_two_name).count()

occurencesDF.sort(occurencesDF["count"].desc()).show(10, False)
occurencesMaxTempDF.sort(occurencesMaxTempDF["count"].desc()).show(10, False)

leastFreq = occurencesDF.orderBy(col("count").asc()).first()
mostFreq = occurencesDF.orderBy(col("count").desc()).first()
leastFreqMaxTemp = occurencesMaxTempDF.orderBy(col("count").asc()).first()
mostFreqMaxTemp = occurencesMaxTempDF.orderBy(col("count").desc()).first()

(
    "leastFreqMinTemp: {} - {}".format(leastFreq[column_name], leastFreq["count"]),
    "mostFreqMinTemp: {} - {}".format(mostFreq[column_name], mostFreq["count"]),
    "leastFreqMaxTemp: {} - {}".format(leastFreqMaxTemp[column_two_name], leastFreqMaxTemp["count"]),
    "mostFreqMaxTemp: {} - {}".format(mostFreqMaxTemp[column_two_name], mostFreqMaxTemp["count"]),
)

Most and least frequent occurrences:
+-------+-----+
|MinTemp|count|
+-------+-----+
|null   |1485 |
|11.0   |899  |
|10.2   |898  |
|9.6    |896  |
|10.5   |884  |
|10.8   |872  |
|9.0    |872  |
|10.0   |871  |
|12.0   |866  |
|8.9    |861  |
+-------+-----+
only showing top 10 rows

+-------+-----+
|MaxTemp|count|
+-------+-----+
|null   |1261 |
|20.0   |885  |
|19.0   |843  |
|19.8   |840  |
|20.4   |834  |
|19.9   |823  |
|20.8   |817  |
|19.5   |812  |
|18.5   |811  |
|21.0   |810  |
+-------+-----+
only showing top 10 rows



('leastFreqMinTemp: -7.099999904632568 - 1',
 'mostFreqMinTemp: None - 1485',
 'leastFreqMaxTemp: 47.0 - 1',
 'mostFreqMaxTemp: None - 1261')

### D. Rain related and other columns basic profiling

In [38]:
analysis_col_list = ["Humidity9am", "Humidity3pm", "Pressure9am", "Pressure3pm", "Cloud9am", "Cloud3pm", "Evaporation", "Sunshine"]

print ("Summary of columns:")
# let's use pandas to have a clearer table
df.select(analysis_col_list).summary().toPandas()

Summary of columns:


,summary,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Evaporation,Sunshine
0,count,142806,140953,130395,130432,89572,86102,82670,75625
1,mean,68.88083133761887,51.5391158755046,1017.6499397947478,1015.2558887915008,4.4474612602152455,4.509930082924903,5.468231521887871,7.611177522303053
2,stddev,19.029164451844167,20.795901656021204,7.106530159387462,7.037413603567235,2.8871588535172408,2.720357310332465,4.193704096708969,3.7854829656916533
3,min,0,0,980.5,977.1,0,0,0.0,0.0
4,25%,57,37,1012.9,1010.4,1,2,2.6,4.8
5,50%,70,52,1017.6,1015.2,5,5,4.8,8.4
6,75%,83,66,1022.4,1020.0,7,7,7.4,10.6
7,max,100,100,1041.0,1039.6,9,9,145.0,14.5


In [39]:
print("Checking for nulls on columns:")
df.select([count(when(col(c).isNull(), c)).alias(c) for c in analysis_col_list]).show()

Checking for nulls on columns:
+-----------+-----------+-----------+-----------+--------+--------+-----------+--------+
|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Evaporation|Sunshine|
+-----------+-----------+-----------+-----------+--------+--------+-----------+--------+
|       2654|       4507|      15065|      15028|   55888|   59358|      62790|   69835|
+-----------+-----------+-----------+-----------+--------+--------+-----------+--------+



In [40]:
print("Checking amount of distinct values:")
df.select([countDistinct(c).alias(c) for c in analysis_col_list]).show()

Checking amount of distinct values:
+-----------+-----------+-----------+-----------+--------+--------+-----------+--------+
|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Evaporation|Sunshine|
+-----------+-----------+-----------+-----------+--------+--------+-----------+--------+
|        101|        101|        546|        549|      10|      10|        358|     145|
+-----------+-----------+-----------+-----------+--------+--------+-----------+--------+



In [41]:
print ("Most and least frequent occurrences:")
column_name = "Humidity9am"
column_two_name = "Humidity3pm"

occurencesDF= df.groupBy(column_name).count()
occurencesHumidity3pmDF= df.groupBy(column_two_name).count()

occurencesDF.sort(occurencesDF["count"].desc()).show(10, False)
occurencesHumidity3pmDF.sort(occurencesHumidity3pmDF["count"].desc()).show(10, False)

leastFreq    = occurencesDF.orderBy(col("count").asc()).first()
mostFreq      = occurencesDF.orderBy(col("count").desc()).first()
leastFreqHumidity3pm = occurencesHumidity3pmDF.orderBy(col("count").asc()).first()
mostFreqHumidity3pm = occurencesHumidity3pmDF.orderBy(col("count").desc()).first()

(
    "leastFreqHumidity9am: {} - {}".format(leastFreq[column_name], leastFreq["count"]),
    "mostFreqHumidity9am: {} - {}".format(mostFreq[column_name], mostFreq["count"]),
    "leastFreqHumidity3pm: {} - {}".format(leastFreqHumidity3pm[column_two_name], leastFreqHumidity3pm["count"]),
    "mostFreqHumidity3pm: {} - {}".format(mostFreqHumidity3pm[column_two_name], mostFreqHumidity3pm["count"]),
)

Most and least frequent occurrences:
+-----------+-----+
|Humidity9am|count|
+-----------+-----+
|99         |3391 |
|70         |3026 |
|69         |3023 |
|65         |3014 |
|68         |3011 |
|71         |2976 |
|66         |2973 |
|67         |2950 |
|74         |2917 |
|72         |2914 |
+-----------+-----+
only showing top 10 rows

+-----------+-----+
|Humidity3pm|count|
+-----------+-----+
|null       |4507 |
|52         |2751 |
|55         |2738 |
|57         |2728 |
|53         |2697 |
|59         |2690 |
|58         |2643 |
|54         |2642 |
|50         |2624 |
|51         |2621 |
+-----------+-----+
only showing top 10 rows



('leastFreqHumidity9am: 0 - 1',
 'mostFreqHumidity9am: 99 - 3391',
 'leastFreqHumidity3pm: 0 - 4',
 'mostFreqHumidity3pm: None - 4507')

### E. Issue related columns basic profiling

In [42]:
analysis_col_list = ["RainTomorrow", "Rainfall", "RainToday"]

print ("Summary of columns:")
df.select(analysis_col_list).summary().show()

Summary of columns:
+-------+------------+------------------+---------+
|summary|RainTomorrow|          Rainfall|RainToday|
+-------+------------+------------------+---------+
|  count|      142193|            142199|   142199|
|   mean|        null|2.3609181508908756|     null|
| stddev|        null|  8.47805974281768|     null|
|    min|          No|               0.0|       No|
|    25%|        null|               0.0|     null|
|    50%|        null|               0.0|     null|
|    75%|        null|               0.8|     null|
|    max|         Yes|             371.0|      Yes|
+-------+------------+------------------+---------+



In [43]:
print("Checking for nulls on columns:")
df.select([count(when(col(c).isNull(), c)).alias(c) for c in analysis_col_list]).show()

Checking for nulls on columns:
+------------+--------+---------+
|RainTomorrow|Rainfall|RainToday|
+------------+--------+---------+
|        3267|    3261|     3261|
+------------+--------+---------+



In [44]:
print("Checking amount of distinct values:")
df.select([countDistinct(c).alias(c) for c in analysis_col_list]).show()

Checking amount of distinct values:
+------------+--------+---------+
|RainTomorrow|Rainfall|RainToday|
+------------+--------+---------+
|           2|     681|        2|
+------------+--------+---------+



RainTomorrow and RainToday can be only True or False so two options

In [45]:
df.filter(df.RainTomorrow == "Yes").count()

31877

In [46]:
df.filter(df.RainToday == "Yes").count()

31880

There may be slight mistakes since these numbers can be different only the last column is True but the difference should be 1 in that case. But the difference is close enough to continue with both columns.

In [47]:
print("RainTomorrow/RainToday unique values")

df.select("RainTomorrow").distinct().show()
df.select("RainToday").distinct().show()

RainTomorrow/RainToday unique values
+------------+
|RainTomorrow|
+------------+
|        null|
|          No|
|         Yes|
+------------+

+---------+
|RainToday|
+---------+
|     null|
|       No|
|      Yes|
+---------+



## 5. Answer some business questions to predict rainfall components

**Rates of rainfall**: [Source](https://water.usgs.gov/edu/activity-howmuchrain-metric.html)<br> 
* Drizzle, very small droplets. <br>
* Slight (fine) drizzle: Detectable as droplets only on the face, car windscreens and windows. <br>
* Moderate drizzle: Windows and other surfaces stream with water. <br>
* Heavy (thick) drizzle: Impairs visibility and is measurable in a raingauge, rates up to 1 mm per hour. <br>
* Rain, drops of appreciable size and may be described as small to large drops. It is possible to have rain drops within drizzle! <br>
* **Slight rain**: Less than 0.5 mm per hour. <br>
* **Moderate rain**: Greater than 0.5 mm per hour, but less than 4.0 mm per hour. <br>
* **Heavy rain**: Greater than 4 mm per hour, but less than 8 mm per hour. <br>
* **Very heavy rain**: Greater than 8 mm per hour. <br>
* Slight shower: Less than 2 mm per hour. <br>
* Moderate shower: Greater than 2 mm, but less than 10 mm per hour. <br>
* Heavy shower: Greater than 10 mm per hour, but less than 50 mm per hour. <br>
* Violent shower: Greater than 50 mm per hour. <br>

### A. Ratio of rainy days vs. non-rainy days and their levels

In [48]:
from pyspark.sql.functions import count, round

#   Delay severity is going to be categorized as follows (see the resource above from water.usgs.gov)
#   Let's assume that it rained for just one hour:
#
#   "very low"   - <= 0.5 mm of Rainfall
#   "low"        - 0.5 to 1.0 mm of Rainfall
#   "medium"     - 1.0 to 4.0 mm of Rainfall # the dataset assumes that the day is rainy if the level is more than 1.0 mm
#   "high"       - 4.0 to 8.0 mm of Rainfall
#   "very high"  - > 8.0 mm of Rainfall

# 1. Let's enrich the DF with rainfall level based on our categorization
totalRainfall = df.count()
rainFallDF = (df
                   .where(col("Rainfall").isNotNull())
                   .withColumn("RainfallLevel", 
                               when(col("Rainfall")<=0.1, "1. very low")
                               .when((col("Rainfall")>0.5) & (col("Rainfall")<=1.0), "2. low")
                               .when((col("Rainfall")>1.0) & (col("Rainfall")<=4.0), "3. medium")
                               .when((col("Rainfall")>4.0) & (col("Rainfall")<=8.0), "4. high")
                               .otherwise("5. very high"))
    .cache() # optimization to make the processing faster
                        )


# 2. Ready to answer to this business question
rainFallDF.select("RainfallLevel")\
                     .groupBy("RainfallLevel")\
                     .agg(count("RainfallLevel").alias("NumDays"), \
                          (count("RainfallLevel")/totalRainfall*100).alias("Ratio"))\
                     .orderBy("RainfallLevel")\
                     .select("RainfallLevel", "NumDays" , round("Ratio", 2).alias("RoundedRatio")).show()

+-------------+-------+------------+
|RainfallLevel|NumDays|RoundedRatio|
+-------------+-------+------------+
|  1. very low|  91080|       62.62|
|       2. low|   6435|        4.42|
|    3. medium|  13459|        9.25|
|      4. high|   7004|        4.82|
| 5. very high|  24221|       16.65|
+-------------+-------+------------+



We can see that most days pass without rain. But also 17% of days with relatively heavy rain.

### B. Compare Humidity, Max/MinTemp, Temperature, Wind Gust Speed of days with different levels of Rainfall

In [49]:
from pyspark.sql.functions import max, min, avg, stddev
from pyspark.sql.types import IntegerType
  
#   Since we had cleaning beforehand, the data is ready for detailed analysis

cachedRainFallDF = (rainFallDF
                       .cache() # optimization to make the processing faster
                 )

display(Markdown("**'Humidity9am'** (percent):")) 
(cachedRainFallDF.groupBy("RainfallLevel")
              .agg(avg("Humidity9am").alias("AverageHumidity9am"),
                   min("Humidity9am").alias("LowestHumidity9am"),
                   max("Humidity9am").alias("HighestHumidity9am"),
                   stddev("Humidity9am").alias("StdDevHumidity9am"))
              .orderBy("RainfallLevel").show())


display(Markdown("**'Humidity3pm'** (percent):")) 
(cachedRainFallDF.groupBy("RainfallLevel")
              .agg(avg("Humidity3pm").alias("AverageHumidity3pm"),
                   min("Humidity3pm").alias("LowestHumidity3pm"),
                   max("Humidity3pm").alias("HighestHumidity3pm"),
                   stddev("Humidity3pm").alias("StdDevHumidity3pm"))
              .orderBy("RainfallLevel").show())


display(Markdown("**'MaxTemp'** (in degrees celsius):")) 
(cachedRainFallDF.groupBy("RainfallLevel")
              .agg(avg("MaxTemp").alias("AverageMaxTemp"),
                   min("MaxTemp").alias("LowestMaxTemp"),
                   max("MaxTemp").alias("HighestMaxTemp"),
                   stddev("MaxTemp").alias("StdDevMaxTemp"))
              .orderBy("RainfallLevel").show())


display(Markdown("**'MinTemp'** (in degrees celsius):")) 
(cachedRainFallDF.groupBy("RainfallLevel")
              .agg(avg("MinTemp").alias("AverageMinTemp"),
                   min("MinTemp").alias("LowestMinTemp"),
                   max("MinTemp").alias("HighestMinTemp"),
                   stddev("MinTemp").alias("StdDevMinTemp"))
              .orderBy("RainfallLevel").show())


display(Markdown("**'WindGustSpeed'** (km/h):")) 
(cachedRainFallDF.groupBy("RainfallLevel")
              .agg(avg("WindGustSpeed").alias("AverageWindGustSpeed"),
                   min("WindGustSpeed").alias("LowestWindGustSpeed"),
                   max("WindGustSpeed").alias("HighestWindGustSpeed"),
                   stddev("WindGustSpeed").alias("StdDevWindGustSpeed"))
              .orderBy("RainfallLevel").show())


display(Markdown("**'Temp9am'** (in degrees celsius):")) 
(cachedRainFallDF.groupBy("RainfallLevel")
              .agg(avg("Temp9am").alias("AverageTemp9am"),
                   min("Temp9am").alias("LowestTemp9am"),
                   max("Temp9am").alias("HighestTemp9am"),
                   stddev("Temp9am").alias("StdDevTemp9am"))
              .orderBy("RainfallLevel").show())


display(Markdown("**'Temp3pm'** (in degrees celsius):")) 
(cachedRainFallDF.groupBy("RainfallLevel")
              .agg(avg("Temp3pm").alias("AverageTemp3pm"),
                   min("Temp3pm").alias("LowestTemp3pm"),
                   max("Temp3pm").alias("HighestTemp3pm"),
                   stddev("Temp3pm").alias("StdDevTemp3pm"))
              .orderBy("RainfallLevel").show())

**'Humidity9am'** (percent):

+-------------+------------------+-----------------+------------------+------------------+
|RainfallLevel|AverageHumidity9am|LowestHumidity9am|HighestHumidity9am| StdDevHumidity9am|
+-------------+------------------+-----------------+------------------+------------------+
|  1. very low| 62.84813842455662|                0|               100|18.540841299718984|
|       2. low| 77.19186504808451|               10|               100|15.285548133662981|
|    3. medium| 79.38014600737563|                1|               100|14.319941752206883|
|      4. high| 81.25076009845084|               14|               100|13.769973690190458|
| 5. very high|  79.8820641913153|                1|               100| 15.01801871857628|
+-------------+------------------+-----------------+------------------+------------------+



**'Humidity3pm'** (percent):

+-------------+------------------+-----------------+------------------+------------------+
|RainfallLevel|AverageHumidity3pm|LowestHumidity3pm|HighestHumidity3pm| StdDevHumidity3pm|
+-------------+------------------+-----------------+------------------+------------------+
|  1. very low|44.857337354006795|                0|               100|19.410347322021217|
|       2. low| 60.77081668531245|                5|               100|17.454898728730374|
|    3. medium|63.119221967963384|                1|               100| 17.18607284132653|
|      4. high| 65.93331370528485|                6|               100| 17.27868124441239|
| 5. very high| 63.36847248778415|                1|               100|18.133107232689323|
+-------------+------------------+-----------------+------------------+------------------+



**'MaxTemp'** (in degrees celsius):

...

+-------------+--------------------+-------------------+--------------------+-------------------+
|RainfallLevel|AverageWindGustSpeed|LowestWindGustSpeed|HighestWindGustSpeed|StdDevWindGustSpeed|
+-------------+--------------------+-------------------+--------------------+-------------------+
|  1. very low|   38.90988876894917|                  7|                 130| 12.761121035053586|
|       2. low|   40.44055237453688|                  6|                 113|  13.92959458685912|
|    3. medium|   41.97314629258517|                  9|                 135| 14.207463267827045|
|      4. high|   43.90278425882719|                  7|                 109| 14.526739930842059|
| 5. very high|   41.71483262108262|                  7|                 135| 15.287392451713098|
+-------------+--------------------+-------------------+--------------------+-------------------+



**'Temp9am'** (in degrees celsius):

+-------------+------------------+-------------+--------------+-----------------+
|RainfallLevel|    AverageTemp9am|LowestTemp9am|HighestTemp9am|    StdDevTemp9am|
+-------------+------------------+-------------+--------------+-----------------+
|  1. very low|17.814084682033865|         -5.3|          40.2|6.530027236431616|
|       2. low|15.454067582736615|         -5.6|          36.8|6.114744534118953|
|    3. medium|15.264453735030349|         -5.5|          35.8|6.008273854494421|
|      4. high| 15.44297830110521|         -7.2|          32.3|6.038622690684605|
| 5. very high|15.677078652778144|         -5.9|          36.0|6.299452768583099|
+-------------+------------------+-------------+--------------+-----------------+



**'Temp3pm'** (in degrees celsius):

+-------------+------------------+-------------+--------------+------------------+
|RainfallLevel|    AverageTemp3pm|LowestTemp3pm|HighestTemp3pm|     StdDevTemp3pm|
+-------------+------------------+-------------+--------------+------------------+
|  1. very low|23.304804204747374|         -4.0|          46.7| 6.806170871279209|
|       2. low|18.829858801394877|         -3.0|          42.6|6.1572952623697805|
|    3. medium| 18.39319402943301|         -4.4|          44.7|  6.13526373501759|
|      4. high| 18.30713137457193|         -5.4|          40.2| 6.271218431201189|
| 5. very high| 19.17177249441582|         -5.1|          43.3| 6.199604232821436|
+-------------+------------------+-------------+--------------+------------------+



### C. Top 20 locations with the highest and lowest levels of rainfall during years

#### A. Top 20 locations with the highest levels of rainfall during years

In [50]:
# Our answer to this business question will be:
#   1. List of top 20 locations in Australia with high level of rainfall during the years
#   2. List of top 20 locations in Australia airports with positive rainfall by level (medium,
#      high and very high >= 1.00 mm)

# In order to be able to deliver these insights, we need some preparation:
#   1. Define a DataFrame with total Rainfall day per Location (totalRainfallLocationDF)
#   2. Define a DataFrame with aggregated data by Location and RainfallLevel to figure out
#      number of days rained per rainfall level (numRainfallFD)
#   3. Combine both DataFrames to come up with one single DataFrame containing total rainfall days
#      per location and number of rainfall days by level to compute ratios (combinedDF)

totalRainfallLocationDF = ( 
            df.groupBy("Location")
            .agg(count(lit(1)).alias("TotalRainfallDays")
            )
)
    
numRainfallDF = ( cachedRainFallDF
                 .where((col("RainfallLevel")!="1. very low") & (col("RainfallLevel")!="2. low"))
                 .select("Location", "RainfallLevel")
                 .groupBy("Location", "RainfallLevel")
                 .agg(count(lit(1)).alias("NumRainfallDays"))
                )

combinedDF = (
  numRainfallDF
     .join(totalRainfallLocationDF, "Location")
     .withColumn("RainfallRatio", round(col("NumRainfallDays")/col("TotalRainfallDays")*100,2))
     .orderBy(col("RainfallRatio").desc())
     .cache()
)

display(Markdown("**Top 20 locations** with the highest rainfall level (**very high**) by ratio (in \%):"))
combinedDF.limit(20).show()

display(Markdown("**Top 20 locations with positive rainfall** by rainfall level (in mm):"))
(combinedDF
   .groupBy("Location")
   .pivot("RainfallLevel")
   .min("RainfallRatio")
   .orderBy(col("`5. very high`").desc(), col("`4. high`").desc(), col("`3. medium`").desc())
   .limit(20).show())

**Top 20 locations** with the highest rainfall level (**very high**) by ratio (in \%):

+-------------+-------------+---------------+-----------------+-------------+
|     Location|RainfallLevel|NumRainfallDays|TotalRainfallDays|RainfallRatio|
+-------------+-------------+---------------+-----------------+-------------+
|     Portland| 5. very high|            787|             3009|        26.15|
|  Witchcliffe| 5. very high|            723|             3009|        24.03|
| MountGambier| 5. very high|            692|             3040|        22.76|
|  MountGinini| 5. very high|            680|             3040|        22.37|
|NorfolkIsland| 5. very high|            667|             3009|        22.17|
|       Cairns| 5. very high|            673|             3040|        22.14|
|      Penrith| 5. very high|            670|             3039|        22.05|
|      Walpole| 5. very high|            660|             3006|        21.96|
|     Dartmoor| 5. very high|            657|             3009|        21.83|
|         Sale| 5. very high|            645|             3009| 

**Top 20 locations with positive rainfall** by rainfall level (in mm):

+-------------+---------+-------+------------+
|     Location|3. medium|4. high|5. very high|
+-------------+---------+-------+------------+
|     Portland|    17.95|   9.24|       26.15|
|  Witchcliffe|     12.0|   6.48|       24.03|
| MountGambier|    15.92|   6.28|       22.76|
|  MountGinini|     9.87|   5.69|       22.37|
|NorfolkIsland|    14.82|   6.28|       22.17|
|       Cairns|    11.12|   5.92|       22.14|
|      Penrith|     8.06|   3.55|       22.05|
|      Walpole|    13.17|   6.85|       21.96|
|     Dartmoor|    15.35|   7.41|       21.83|
|         Sale|     11.5|   4.65|       21.44|
|    NorahHead|     9.29|   6.13|       21.07|
| CoffsHarbour|    10.04|   5.15|        20.8|
|     Watsonia|    12.13|   5.75|       20.41|
|       Sydney|     8.85|   5.95|       20.39|
|      Bendigo|     7.93|   4.24|       19.67|
|     Richmond|     7.28|   3.99|       19.21|
|       Darwin|     6.55|   4.67|       19.14|
|       Hobart|    12.84|   5.26|       18.95|
|    Newcastl

#### B. Top 20 locations with the lowest levels of rainfall during years

In [51]:
# Our answer to this business question will be:
#   1. List of top 20 locations in Australia with very low level of rainfall during the years
#   2. List of top 20 locations in Australia airports with negative rainfall by level (low,
#      very low < 1.00 mm)

# In order to be able to deliver these insights, we need some preparation:
#   1. Define a DataFrame with total Rainfall day per Location (totalRainfallLocationDF)
#   2. Define a DataFrame with aggregated data by Location and RainfallLevel to figure out
#      number of days rained per rainfall level (numRainfallFD)
#   3. Combine both DataFrames to come up with one single DataFrame containing total rainfall days
#      per location and number of rainfall days by level to compute ratios (combinedDF)

totalRainfallLocationDF = ( 
            df.groupBy("Location")
            .agg(count(lit(1)).alias("TotalRainfallDays")
            )
)
    
numRainfallFD = ( cachedRainFallDF
                 .where((col("RainfallLevel")!="5. very high") & (col("RainfallLevel")!="4. high") & (col("RainfallLevel")!="3. medium"))
                 .select("Location", "RainfallLevel")
                 .groupBy("Location", "RainfallLevel")
                 .agg(count(lit(1)).alias("NumRainfallDays"))
                )

combinedDF = (
  numRainfallFD
     .join(totalRainfallLocationDF, "Location")
     .withColumn("RainfallRatio", round(col("NumRainfallDays")/col("TotalRainfallDays")*100,2))
     .orderBy(col("RainfallRatio").desc())
     .cache()
    )

display(Markdown("**Top 20 locations** with the lowest rainfall level (**very low**) by ratio (in \%):"))
combinedDF.limit(20).show()
display(Markdown("**Top 20 locations with negative rainfall** by rainfall level (in mm):"))
(combinedDF
   .groupBy("Location")
   .pivot("RainfallLevel")
   .min("RainfallRatio")
   .orderBy(col("`1. very low`").desc(), col("`2. low`").desc())
   .limit(20).show())

**Top 20 locations** with the lowest rainfall level (**very low**) by ratio (in \%):

+-------------+-------------+---------------+-----------------+-------------+
|     Location|RainfallLevel|NumRainfallDays|TotalRainfallDays|RainfallRatio|
+-------------+-------------+---------------+-----------------+-------------+
| AliceSprings|  1. very low|           2665|             3040|        87.66|
|      Woomera|  1. very low|           2603|             3009|        86.51|
|        Uluru|  1. very low|           1324|             1578|         83.9|
|      Mildura|  1. very low|           2424|             3009|        80.56|
|        Cobar|  1. very low|           2382|             3009|        79.16|
|    Katherine|  1. very low|           1229|             1578|        77.88|
|        Moree|  1. very low|           2267|             3009|        75.34|
| PerthAirport|  1. very low|           2235|             3009|        74.28|
|   Townsville|  1. very low|           2232|             3040|        73.42|
|        Perth|  1. very low|           2293|             3193| 

**Top 20 locations with negative rainfall** by rainfall level (in mm):

+-------------+-----------+------+
|     Location|1. very low|2. low|
+-------------+-----------+------+
| AliceSprings|      87.66|  1.68|
|      Woomera|      86.51|  1.69|
|        Uluru|       83.9|  1.27|
|      Mildura|      80.56|  2.89|
|        Cobar|      79.16|  2.69|
|    Katherine|      77.88|  2.47|
|        Moree|      75.34|  2.36|
| PerthAirport|      74.28|  3.02|
|   Townsville|      73.42|  3.52|
|        Perth|      71.81|  2.94|
|   WaggaWagga|      71.55|  4.02|
|  Tuggeranong|      70.71|  3.13|
|     Canberra|      70.46|  3.52|
|   PearceRAAF|      68.73|  2.49|
|   SalmonGums|      67.74|   4.5|
|       Darwin|      67.24|  2.41|
|         Nhil|      66.67|  4.44|
|       Albury|      66.28|  3.62|
|    Nuriootpa|      64.57|  5.05|
|BadgerysCreek|      64.44|  3.59|
+-------------+-----------+------+



## Conclusions
* The mean rainfall in all regions is 2.36 mm which is above the threshold of 1 mm. This must be a result of the rainfall days with very high level.
* The rainfall level has a higher correlation with min/max temperature during the day and humidity measured at 3 pm and 9 am.
* Minimum temperature, maximum temperature, wind gust speed, and humidity at 3 pm and 9 am components are remarkably different among the days with very low and very high levels of rainfall.
* Portland, Witchcliffe, MountGambier, MountGinini, NorfolkIsland, Cairns, Penrith, Walpole, Dartmoor, Sale are the top 10 regions with very high levels of rainfall.
* AliceSprings, Woomera, Uluru, Mildura, Cobar, Katherine, Moree, PerthAirport, Townsville, Perth are the top 10 regions with very low levels of rainfall.

#### Result
* Authorities can focus on strategies to improve the amount of average daily rainfall levels in AliceSprings, Woomera, Uluru, Mildura, Cobar, Katherine, Moree, PerthAirport, Townsville, Perth.
* While implementing new strategies, improvement can be measured by components such as average minimum temperature, average maximum temperature, average wind gust speed, and average humidity at 3 pm and 9 am.